In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

np.random.seed(0)

X = np.random.rand(100, 1)
y = 2 * X + 1 + 0.1 * np.random.randn(100, 1)

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

class Config():
    def __init__(self):
        self.embed_dim = 128
        self.num_heads = 128
        self.head_dim = 128
        self.hidden_size = 128
        self.input_size = 128
        self.output_size = 1


class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()  
        self.fc2 = nn.Linear(hidden_size, output_size)  
        # self.sigmoid = nn.Sigmoid()  

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        # x = self.sigmoid(x)
        return x


class CompareSearchModel(nn.Module):
    def __init__(self, cfg):
        super(CompareSearchModel, self).__init__()

        self.num_heads = cfg.num_heads
        self.head_dim = cfg.head_dim
        self.mha = nn.MultiheadAttention(cfg.embed_dim, cfg.num_heads)
        self.mlp = MLP(cfg.input_size, cfg.hidden_size, cfg.output_size)


    def forward(self, embed_image, semantic_features):

        flattened_embed = embed_image.view(embed_image.size(0), -1)
        concatenated_features = torch.cat((flattened_embed, semantic_features), dim=1)
        x = concatenated_features.view(concatenated_features.shape[0], -1, self.num_heads, self.head_dim)
        attn_output, _ = self.mha(x,x,x)
        logits = self.mlp(x)

        return logits


cfg = Config()
model = CompareSearchModel(cfg)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

num_epochs = 100
for epoch in range(num_epochs):
    outputs = model(X)
    loss = criterion(outputs, y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 100 == 0:
        print(f'Época [{epoch + 1}/{num_epochs}], Perda: {loss.item():.4f}')

print('Training Completed')

with torch.no_grad():
    test_input = torch.tensor([[0.6]], dtype=torch.float32)
    predicted_output = model(test_input)
    print(f'Predição para entrada 0.6: {predicted_output.item():.4f}')

TypeError: CompareSearchModel.forward() missing 1 required positional argument: 'semantic_features'